# 🚀 Vorlage für interaktive Simulationen mit Jupyter Notebook

Diese Vorlage enthält alle notwendigen Komponenten für interaktive Simulationen mit Schiebereglern, einem Button zur Neuberechnung sowie ein Kontrollkästchen. Fügen Sie einfach Ihre eigene Simulation in die dafür vorgesehene Funktion ein!

## 📌 Wichtiger Hinweis:
Bevor Sie mit der Simulation beginnen, klicken Sie bitte auf:
**🔄 Restart Kernel and Run All Cells** (in der Jupyter-Werkzeugleiste oben). Dies sorgt dafür, dass alle interaktiven Elemente korrekt funktionieren.

Je nach Betriebssystem kann das Symbol unterschiedlich aussehen:
- macOS: Zwei Dreiecke ähnlich dem Schnellvorlauf-Button ⏩
- Andere Systeme: Zwei kreisförmige Pfeile 🔄

> 📍 **Warum?** Dadurch wird sichergestellt, dass alle Zellen in der richtigen Reihenfolge ausgeführt werden und die interaktiven Elemente korrekt funktionieren.
---

## 🛠 Anforderungen (falls die Datei lokal genutzt wird)

Falls Sie das Notebook direkt auf Ihrem Rechner ausführen möchten, benötigen Sie:

- **Python 3.7 oder höher**
- **Jupyter Notebook** (oder JupyterLab)
- **Notwendige Python-Pakete**, die im `requirements.txt` aufgeführt sind:
  - `numpy`
  - `matplotlib`
  - `scipy`
  - `ipywidgets`
  - `ipython`
  - `notebook`

### Installation der Pakete:
Nutzen Sie den folgenden Befehl, um alle notwendigen Pakete zu installieren:

```bash
pip install -r requirements.txt
```

Falls Sie die Datei **direkt über Binder** nutzen, müssen Sie keine zusätzliche Software installieren.

---
## 📜 Lizenz
Dieses Projekt steht unter der [MIT-Lizenz](LICENSE).

---

In [2]:
# 📌 Notwendige Bibliotheken importieren
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import ipywidgets as widgets
from IPython.display import display


## 🔧 Interaktive Steuerelemente
Die folgenden Schieberegler und Auswahlfelder ermöglichen die Steuerung der Simulation:


In [15]:
# 📊 Schieberegler für die Parameter der Simulation
n_slider = widgets.IntSlider(min=10, max=500,
                             step=10, value=50,
                             description='n:')
p_slider = widgets.FloatSlider(min=0.1, max=0.9, 
                               step=0.05, value=0.6,
                               description='p:')
num_simulations_slider = widgets.IntSlider(min=10, max=500, 
                                           step=10, value=100,
                                           description='Anzahl:')
confidence_slider = widgets.FloatSlider(min=0.8, max=0.99, 
                                        step=0.01, value=0.95,
                                        description='CI-Level:')
method_dropdown = widgets.Dropdown(options=['WALD', 'WILSON'],
                                   value='WALD',
                                   description='Methode:')
show_pi_intervalls = widgets.Checkbox(value=True, 
                                      description='PI von p zeigen')

## 🏗 Simulation einfügen
Hier fügen Sie Ihre eigene Simulation ein. Standardmäßig ist eine Simulation von **Konfidenzintervallen** enthalten.


In [32]:
# Funktion zur Berechnung des WALD-Konfidenzintervalls
def wald_ci(p_hat, n, z):
    se = np.sqrt(p_hat * (1 - p_hat) / n)
    ci_lower = p_hat - z * se
    ci_upper = p_hat + z * se
    return ci_lower, ci_upper

# Funktion zur Berechnung des WILSON-Konfidenzintervalls
def wilson_ci(p_hat, n, z):
    denominator = 1 + z**2 / n
    center = (p_hat + (z**2 / (2 * n))) / denominator
    margin = (z / denominator) * np.sqrt((p_hat * (1 - p_hat) / n) + (z**2 / (4 * n**2)))
    ci_lower = center - margin
    ci_upper = center + margin
    return ci_lower, ci_upper


def simulate(n, p, num_simulations, confidence, method, show_pi):
    z = norm.ppf(1 - (1 - confidence) / 2)
    miss_count = 0
    
    plt.figure(figsize=(8, 10))
    
    for i in range(num_simulations):
        sample = np.random.binomial(n, p) / n
        
        # Berechnung der Konfidenzintervalle über die definierten Funktionen
        if method == 'WALD':
            ci_lower, ci_upper = wald_ci(sample, n, z)
        else:
            ci_lower, ci_upper = wilson_ci(sample, n, z)
        
        # Bestimmen der Farbe
        color = 'blue' if ci_lower <= p <= ci_upper else 'red'
        if color == 'red':
            miss_count += 1
        
        # Zeichne das Intervall
        plt.plot([ci_lower, ci_upper], [i, i], color=color, lw=2)
        
        # Punkt h und PI-Grenzen anzeigen (falls ausgewählt)
        if show_pi:
            plt.plot(sample, i, 'ko', markersize=4)
    
    # Wahre Parametergrenzen
    plt.axvline(p, color='black', linestyle='--', label='Wahrer Parameter')
    if show_pi:
        se_p = np.sqrt(p * (1 - p) / n)
        prog_lower, prog_upper = p - z * se_p, p + z * se_p
        plt.axvline(prog_lower, color='gray', linestyle='dotted',
                    label='Prognose-Intervall')
        plt.axvline(prog_upper, color='gray', linestyle='dotted')
    
    plt.title(f'Simulation von {num_simulations} Konfidenzintervallen \
                ({method}-Methode)\nFehlerrate: {miss_count}/{num_simulations} \
                ({(miss_count/num_simulations)*100:.2f}%)')
    plt.xlabel('p',fontsize=16)
    plt.ylabel('Simulationen',fontsize=16)
    plt.legend()
    plt.xlim(0,1) # damit kann man Abhängigkeiten besser erkennen
    plt.show()

## 🎛 Steuerung der Simulation
Hier wird die Simulation mit den interaktiven Elementen verbunden.


In [34]:
# Button zur Aktualisierung der Simulation
button = widgets.Button(description='Neue Simulation')
output = widgets.Output()

def on_button_clicked(b):
    with output:
        output.clear_output(wait=True)
        simulate(n_slider.value, p_slider.value, 
                 num_simulations_slider.value, confidence_slider.value,
                 method_dropdown.value, show_pi_intervalls.value)

button.on_click(on_button_clicked)

# Anzeige der Steuerelemente und erste Simulation starten
ui = widgets.VBox([n_slider, p_slider, 
                   num_simulations_slider, confidence_slider,
                   method_dropdown, show_pi_intervalls, button])
display(ui, output)

# Starte die erste Simulation automatisch
on_button_clicked(None)

Output()